# 03 — Entrenamiento y Evaluación (SUPERVISED y CONTINUAL con EWC/NAIVE)

Este notebook entrena un modelo **SNN** para **regresión del ángulo de dirección (steering)** en dos protocolos:

- **Supervised** sobre `circuito1`.
- **Continual** con dos tareas secuenciales `circuito1 → circuito2` usando:
  - **EWC** (consolidación elástica de pesos), o
  - **NAIVE** (baseline sin penalización; equivalente a λ=0).

> **Requisitos previos**: Ejecuta `01_DATA_QC_PREP.ipynb` para generar `train/val/test.csv` y `tasks.json`.


In [ ]:
# =============================================================================
# Imports y setup
# =============================================================================
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

from pathlib import Path
import sys, json, torch

ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.utils import set_seeds, load_preset
from src.datasets import ImageTransform, AugmentConfig
from src.models import build_model, default_tfm_for_model
from src.training import TrainConfig
from src.eval import eval_loader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

MODEL_NAME = "pilotnet_snn"   # "snn_vision" | "pilotnet_ann" | "pilotnet_snn"

# tfm por defecto según el modelo:
tfm = default_tfm_for_model(MODEL_NAME, to_gray=True)

def make_model_fn(tfm):
    # kwargs solo necesarios para pilotnet_snn; ignorados por otros
    return build_model(MODEL_NAME, tfm, beta=0.9, threshold=0.5)

torch.set_num_threads(4)
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")


In [3]:
GPU_ENCODE = True

SAFE_MODE = False
NUM_WORKERS    = 12
PREFETCH       = 2
PIN_MEMORY     = True
PERSISTENT     = True

AUG_CFG_LIGHT = AugmentConfig(prob_hflip=0.5, brightness=None, gamma=None, noise_std=0.0)
AUG_CFG_FULL  = AugmentConfig(prob_hflip=0.5, brightness=(0.9, 1.1), gamma=(0.95, 1.05), noise_std=0.005)
AUG_CFG = AUG_CFG_LIGHT

USE_OFFLINE_BALANCED = True
USE_ONLINE_BALANCING = False

if SAFE_MODE:
    NUM_WORKERS = 0
    PREFETCH = None
    PIN_MEMORY = False
    PERSISTENT = False
    USE_OFFLINE_BALANCED = False
    USE_ONLINE_BALANCING = False
    AUG_CFG = None

print(f"[SAFE_MODE={SAFE_MODE}] workers={NUM_WORKERS} prefetch={PREFETCH} pin={PIN_MEMORY} persistent={PERSISTENT}")


[SAFE_MODE=False] workers=12 prefetch=2 pin=True persistent=True


In [4]:
# =============================================================================
# Verificación de datos (normal y, si existe, balanceado offline)
# =============================================================================
from pathlib import Path

RAW  = ROOT/"data"/"raw"/"udacity"
PROC = ROOT/"data"/"processed"

RUNS = ["circuito1","circuito2"]  # ajusta si hace falta

missing = []
for run in RUNS:
    base = PROC / run

    # Comprobación obligatoria: splits normales
    for part in ["train","val","test"]:
        p = base / f"{part}.csv"
        if not p.exists():
            missing.append(str(p))

    # Comprobación opcional: train_balanced.csv (para modo OFFLINE balanceado)
    p_bal = base / "train_balanced.csv"
    if p_bal.exists():
        print(f"✓ {p_bal} OK")
    else:
        print(f"⚠️  Falta {p_bal}. Si más abajo pones USE_OFFLINE_BALANCED=True, "
              f"ejecuta 01A_PREP_BALANCED.ipynb o el script tools/make_splits_balanced.py")

if missing:
    raise FileNotFoundError(
        "Faltan CSV obligatorios (ejecuta 01A_PREP_BALANCED.ipynb o tu pipeline de prep):\n"
        + "\n".join(" - " + m for m in missing)
    )

print("OK: splits 'train/val/test' encontrados.")


✓ /home/cesar/proyectos/TFM_SNN/data/processed/circuito1/train_balanced.csv OK
✓ /home/cesar/proyectos/TFM_SNN/data/processed/circuito2/train_balanced.csv OK
OK: splits 'train/val/test' encontrados.


In [5]:
# ===================== Balanceo: helper =====================
print(
    "Modo balanceo:",
    "OFFLINE (tasks_balanced.json)" if USE_OFFLINE_BALANCED else "ORIGINAL (tasks.json)",
    "| Balanceo ONLINE:", USE_ONLINE_BALANCING
)

# Seguridad anti doble balanceo:
if USE_OFFLINE_BALANCED and USE_ONLINE_BALANCING:
    raise RuntimeError("Doble balanceo detectado: OFFLINE y ONLINE a la vez. "
                       "Pon USE_ONLINE_BALANCING=False cuando uses train_balanced.csv.")

from pathlib import Path  # (omite esta línea si ya importaste Path arriba)

def _balance_flag(train_csv_path: str | Path) -> bool:
    """
    Activa balanceo ONLINE solo si:
    - USE_ONLINE_BALANCING == True
    - Y el CSV de train NO es 'train_balanced.csv'
    """
    is_balanced_csv = Path(train_csv_path).name == "train_balanced.csv"
    return bool(USE_ONLINE_BALANCING and not is_balanced_csv)


Modo balanceo: OFFLINE (tasks_balanced.json) | Balanceo ONLINE: False


In [6]:
import src.training as training

In [7]:
# =============================================================================
# Elegir split: normal (tasks.json) o balanceado offline (tasks_balanced.json)
# =============================================================================
with open(PROC / ("tasks_balanced.json" if USE_OFFLINE_BALANCED else "tasks.json"), "r", encoding="utf-8") as f:
    tasks_json = json.load(f)

task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]

# Vista rápida: muestra el CSV de train que se usará por cada tarea
print("Tareas y su TRAIN CSV:")
for t in task_list:
    print(f" - {t['name']}: {Path(t['paths']['train']).name}")

task_list[:2]  # vista rápida

# Guardarraíl extra: si has activado el OFFLINE balanceado,
# exige que el 'train' sea train_balanced.csv y que exista.
if USE_OFFLINE_BALANCED:
    for t in task_list:
        train_path = Path(t["paths"]["train"])
        if train_path.name != "train_balanced.csv":
            raise RuntimeError(
                f"[{t['name']}] Esperaba 'train_balanced.csv' pero encontré '{train_path.name}'. "
                "Repite 01A_PREP_BALANCED.ipynb o ajusta USE_OFFLINE_BALANCED=False."
            )
        if not train_path.exists():
            raise FileNotFoundError(
                f"[{t['name']}] No existe {train_path}. Genera los balanceados con 01A_PREP_BALANCED.ipynb."
            )
    print("✔ Verificación OFFLINE balanceado superada (train_balanced.csv por tarea).")


Tareas y su TRAIN CSV:
 - circuito1: train_balanced.csv
 - circuito2: train_balanced.csv
✔ Verificación OFFLINE balanceado superada (train_balanced.csv por tarea).


In [8]:
from pathlib import Path
from src.utils import make_loaders_from_csvs

def make_loader_fn(task, batch_size, encoder, T, gain, tfm, seed, **dl_kwargs):
    RAW = ROOT / "data" / "raw" / "udacity" / task["name"]
    paths = task["paths"]

    # Si vamos a codificar en GPU, pedimos 4D (image) al loader;
    # si no, dejamos el encoder temporal en el propio dataset.
    encoder_for_loader = "image" if (GPU_ENCODE and encoder in {"rate","latency","raw"}) else encoder

    return make_loaders_from_csvs(
        base_dir=RAW,
        train_csv=Path(paths["train"]),
        val_csv=Path(paths["val"]),
        test_csv=Path(paths["test"]),
        batch_size=batch_size,
        encoder=encoder_for_loader,
        T=T, gain=gain, tfm=tfm, seed=SEED,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        persistent_workers=PERSISTENT,
        prefetch_factor=PREFETCH,
        # online balancing opcional:
        aug_train=AUG_CFG,
        balance_train=(USE_ONLINE_BALANCING and Path(paths["train"]).name != "train_balanced.csv"),
        balance_bins=21,
        balance_smooth_eps=1e-3,
    )


In [9]:
from src.runner import run_continual

In [9]:
# === Activar métrica de it/s por época (parche temporal) ===
import time, json
from pathlib import Path
import torch
from torch import nn, optim
from torch.amp import autocast, GradScaler
import src.training as training
from src.utils import set_seeds  # ya lo tienes importado en el notebook

# Guarda la referencia al original para poder restaurar luego
orig_train_supervised = training.train_supervised

def train_supervised_ips(model: nn.Module, train_loader, val_loader, loss_fn: nn.Module,
                         cfg, out_dir: Path, method=None):
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
    if cfg.seed is not None:
        set_seeds(cfg.seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    opt = optim.Adam(model.parameters(), lr=cfg.lr)

    use_amp = bool(cfg.amp and torch.cuda.is_available())
    scaler = GradScaler(enabled=use_amp)

    history = {"train_loss": [], "val_loss": []}
    t0_total = time.perf_counter()

    for epoch in range(1, cfg.epochs + 1):
        model.train()
        running = 0.0
        nb = 0
        t_epoch0 = time.perf_counter()

        for x, y in train_loader:
            # encode/permutación runtime y subida a device (usa tu helper actual)
            x = training._permute_if_needed(x).to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

            opt.zero_grad(set_to_none=True)
            with autocast("cuda", enabled=use_amp):
                y_hat = model(x)
                loss = loss_fn(y_hat, y)
                if method is not None:
                    loss = loss + method.penalty()

            if use_amp:
                scaler.scale(loss).backward()
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(opt); scaler.update()
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                opt.step()

            running += loss.item()
            nb += 1

        epoch_time = time.perf_counter() - t_epoch0
        ips = nb / epoch_time if epoch_time > 0 else float("nan")
        print(f"[TRAIN it/s] epoch {epoch}/{cfg.epochs}: {ips:.1f} it/s  "
              f"({nb} iters en {epoch_time:.2f}s)")

        train_loss = running / max(1, nb)

        # --- validación ---
        model.eval()
        v_running = 0.0; nvb = 0
        with torch.no_grad():
            for x, y in val_loader:
                x = training._permute_if_needed(x).to(device, non_blocking=True)
                y = y.to(device, non_blocking=True)
                with autocast("cuda", enabled=use_amp):
                    y_hat = model(x)
                    v_loss = loss_fn(y_hat, y)
                v_running += v_loss.item(); nvb += 1
        val_loss = v_running / max(1, nvb)
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)

    elapsed = time.perf_counter() - t0_total
    manifest = {
        "epochs": cfg.epochs, "batch_size": cfg.batch_size, "lr": cfg.lr,
        "amp": cfg.amp, "seed": cfg.seed, "elapsed_sec": elapsed,
        "device": str(device), "history": history,
    }
    (out_dir / "manifest.json").write_text(json.dumps(manifest, indent=2), encoding="utf-8")
    return history

# Activa el parche
training.train_supervised = train_supervised_ips
print("✅ it/s por época ACTIVADO. Para desactivarlo: training.train_supervised = orig_train_supervised")

✅ it/s por época ACTIVADO. Para desactivarlo: training.train_supervised = orig_train_supervised


In [10]:
# ====================== CONFIG POR DEFECTO PARA LAS COMPARATIVAS ======================
# Recomendaciones acordadas:
# - fast: EWC λ=1e9 (estable). Extra: λ=1e8 (mejor T2, algo más de olvido)
# - std : EWC λ=1e7 (baseline estable). Extra: λ=3e7 (mejor T2, más olvido)

# EWC_DEFAULTS = {
#     "fast":     {"primary": [1e9, 3e8],      "extra": [3e9]},
#     "std": {"primary": [7e8, 1e9, 1.2e9, 1.5e9], "extra": []},
#     "accurate": {"primary": [1e6, 3e6, 1e7], "extra": []},  # ← mucho más bajo que 1e9
# }

EWC_DEFAULTS = {
    "fast": {"primary": [7e8, 1e9, 1.2e9], "extra": []},
    "std":  {"primary": [5e8, 7e8, 1e9],   "extra": []},
    "accurate": {"primary": [1e6, 3e6, 1e7], "extra": []}, 
    # de momento accurate solo NAIVE; EWC en accurate lo aparcamos
    # "accurate": {"primary": [], "extra": []},
}

INCLUDE_NAIVE    = True          # añade baseline sin EWC
INCLUDE_EXTRAS   = False          # activa los λ "extra" por preset
# SEEDS            = [42]  # multisemillas para medias/σ
SEEDS            = [42, 43, 44]  # multisemillas para medias/σ
ENCODERS         = ["rate"]      # luego podrás añadir "latency"
# FISHER_BY_PRESET = {"fast": 200, "std": 600, "accurate": 600}  # estabiliza el cálculo de Fisher
# FISHER_BY_PRESET = {"fast": 800, "std": 1000}  # estabiliza el cálculo de Fisher
# Endurecemos Fisher para estabilidad; accurate necesita más por ser más largo
# FISHER_BY_PRESET = {"fast": 800, "std": 1200, "accurate": 1500}
FISHER_BY_PRESET = {"fast": 800, "std": 1000, "accurate": 1500}
# Elige qué presets lanzar
PRESETS_TO_RUN = ["fast", "std", "accurate"]  # añade "accurate" si lo necesitas más adelante
# PRESETS_TO_RUN = ["fast"]  # añade "accurate" si lo necesitas más adelante

# ---- Construcción del plan de ejecuciones ----
runs_plan = []
for preset_i in PRESETS_TO_RUN:
    # EWC primary
    for lam in EWC_DEFAULTS[preset_i]["primary"]:
        runs_plan.append((preset_i, "ewc", lam))
    # EWC extras (opcionales)
    if INCLUDE_EXTRAS:
        for lam in EWC_DEFAULTS[preset_i]["extra"]:
            runs_plan.append((preset_i, "ewc", lam))
    # Baseline sin EWC
    if INCLUDE_NAIVE:
        runs_plan.append((preset_i, "naive", None))

print("Plan de runs (preset, método, λ):")
for preset_i, method_i, lam_i in runs_plan:
    print(f"  {preset_i:>7}  {method_i:>5}  λ={lam_i}")
print("Semillas:", SEEDS, " | Encoders:", ENCODERS)
print("Fisher batches por preset:", FISHER_BY_PRESET)


Plan de runs (preset, método, λ):
     fast    ewc  λ=700000000.0
     fast    ewc  λ=1000000000.0
     fast    ewc  λ=1200000000.0
     fast  naive  λ=None
      std    ewc  λ=500000000.0
      std    ewc  λ=700000000.0
      std    ewc  λ=1000000000.0
      std  naive  λ=None
  accurate    ewc  λ=1000000.0
  accurate    ewc  λ=3000000.0
  accurate    ewc  λ=10000000.0
  accurate  naive  λ=None
Semillas: [42, 43, 44]  | Encoders: ['rate']
Fisher batches por preset: {'fast': 800, 'std': 1000, 'accurate': 1500}


In [ ]:
# ====================== DRIVER MULTISEMILLAS (usa la CONFIG de arriba) ======================
for enc in ENCODERS:
    for seed in SEEDS:
        for preset_i, method_i, lam_i in runs_plan:
            print(f"\n=== RUN: preset={preset_i} | method={method_i} | λ={lam_i} | seed={seed} | encoder={enc} ===")
            out_path, _ = run_continual(
                task_list=task_list,
                make_loader_fn=make_loader_fn,
                make_model_fn=make_model_fn,
                tfm=tfm,
                preset=preset_i,
                method=method_i,
                lam=(lam_i if method_i == "ewc" else None),
                seed=seed,
                encoder=enc,
                fisher_batches_by_preset=FISHER_BY_PRESET,
                runtime_encode=GPU_ENCODE,      # << importante
                out_root=ROOT/"outputs",
                verbose=True,
            )
            print("OK:", out_path)

print("\n Listo. Ejecuta las celdas de resumen.")


=== RUN: preset=fast | method=ewc | λ=700000000.0 | seed=42 | encoder=rate ===

--- Tarea 1/2: circuito1 | preset=fast | method=ewc | λ=700000000.0 | B=64 T=10 AMP=True | enc=rate ---
  loader batch shape: (64, 1, 80, 160) | y: (64, 1)
  runtime encode: ON (GPU)


  runtime encode: OFF

--- Tarea 2/2: circuito2 | preset=fast | method=ewc | λ=700000000.0 | B=64 T=10 AMP=True | enc=rate ---
  loader batch shape: (64, 1, 80, 160) | y: (64, 1)
  runtime encode: ON (GPU)


  runtime encode: OFF
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_7e+08_rate_seed_42

=== RUN: preset=fast | method=ewc | λ=1000000000.0 | seed=42 | encoder=rate ===

--- Tarea 1/2: circuito1 | preset=fast | method=ewc | λ=1000000000.0 | B=64 T=10 AMP=True | enc=rate ---
  loader batch shape: (64, 1, 80, 160) | y: (64, 1)
  runtime encode: ON (GPU)


  runtime encode: OFF

--- Tarea 2/2: circuito2 | preset=fast | method=ewc | λ=1000000000.0 | B=64 T=10 AMP=True | enc=rate ---
  loader batch shape: (64, 1, 80, 160) | y: (64, 1)
  runtime encode: ON (GPU)


  runtime encode: OFF
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_1e+09_rate_seed_42

=== RUN: preset=fast | method=ewc | λ=1200000000.0 | seed=42 | encoder=rate ===

--- Tarea 1/2: circuito1 | preset=fast | method=ewc | λ=1200000000.0 | B=64 T=10 AMP=True | enc=rate ---
  loader batch shape: (64, 1, 80, 160) | y: (64, 1)
  runtime encode: ON (GPU)


...

  runtime encode: OFF
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_1e+09_rate_seed_44

=== RUN: preset=fast | method=ewc | λ=1200000000.0 | seed=44 | encoder=rate ===

--- Tarea 1/2: circuito1 | preset=fast | method=ewc | λ=1200000000.0 | B=64 T=10 AMP=True | enc=rate ---
  loader batch shape: (64, 1, 80, 160) | y: (64, 1)
  runtime encode: ON (GPU)


  runtime encode: OFF

--- Tarea 2/2: circuito2 | preset=fast | method=ewc | λ=1200000000.0 | B=64 T=10 AMP=True | enc=rate ---
  loader batch shape: (64, 1, 80, 160) | y: (64, 1)
  runtime encode: ON (GPU)


  runtime encode: OFF
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_1e+09_rate_seed_44

=== RUN: preset=fast | method=naive | λ=None | seed=44 | encoder=rate ===

--- Tarea 1/2: circuito1 | preset=fast | method=naive | λ=- | B=64 T=10 AMP=True | enc=rate ---
  loader batch shape: (64, 1, 80, 160) | y: (64, 1)
  runtime encode: ON (GPU)


  runtime encode: OFF

--- Tarea 2/2: circuito2 | preset=fast | method=naive | λ=- | B=64 T=10 AMP=True | enc=rate ---
  loader batch shape: (64, 1, 80, 160) | y: (64, 1)
  runtime encode: ON (GPU)


  runtime encode: OFF
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_naive_rate_seed_44

=== RUN: preset=std | method=ewc | λ=500000000.0 | seed=44 | encoder=rate ===

--- Tarea 1/2: circuito1 | preset=std | method=ewc | λ=500000000.0 | B=56 T=16 AMP=True | enc=rate ---
  loader batch shape: (56, 1, 80, 160) | y: (56, 1)
  runtime encode: ON (GPU)


In [10]:
# =============================================================================
# Resumen comparativo de todos los continual_* en outputs/
# =============================================================================
import re, json
from pathlib import Path
import pandas as pd

def parse_exp_name(name: str):
    """
    Extrae preset, método, lambda, encoder y seed del nombre de carpeta:

      continual_<preset>_<method>[_lam_<lambda>]_<_encoder>[_seed_<seed>]

    Ejemplos:
      continual_fast_naive_rate_seed_42
      continual_fast_ewc_lam_1e+08_rate_seed_42
      continual_std_ewc_lam_3e+07_latency_seed_43
    """
    m = re.match(
        r"continual_(?P<preset>\w+)_(?P<method>ewc|naive)"
        r"(?:_lam_(?P<lambda>[^_]+))?_(?P<enc>[^_]+)"
        r"(?:_seed_(?P<seed>\d+))?$",
        name
    )
    meta = {"preset": None, "method": None, "lambda": None, "encoder": None, "seed": None}
    if m:
        d = m.groupdict()
        meta.update({
            "preset": d["preset"],
            "method": d["method"],
            "lambda": d.get("lambda"),
            "encoder": d.get("enc"),
            "seed": d.get("seed"),
        })
    return meta

rows = []
root_out = ROOT / "outputs"

for exp_dir in sorted(root_out.glob("continual_*")):
    name = exp_dir.name
    meta = parse_exp_name(name)

    # Saltar nombres no reconocidos (runs muy antiguos)
    if meta["preset"] is None:
        continue

    results_path = exp_dir / "continual_results.json"
    if not results_path.exists():
        continue

    with open(results_path, "r", encoding="utf-8") as f:
        res = json.load(f)

    # Detectar tareas: la "última" es la que NO tiene claves 'after_*'
    task_names = list(res.keys())
    if len(task_names) < 2:
        continue

    def is_last(d):  # no tiene after_*
        return not any(k.startswith("after_") for k in d.keys())

    last_task = None
    first_task = None
    for tn in task_names:
        if is_last(res[tn]):
            last_task = tn
        else:
            first_task = tn

    # Fallback por si no se identifica bien
    if first_task is None or last_task is None:
        task_names_sorted = sorted(task_names)
        first_task = task_names_sorted[0]
        last_task  = task_names_sorted[-1]

    c1, c2 = first_task, last_task

    c1_test_mae = float(res[c1].get("test_mae", float("nan")))
    c2_test_mae = float(res[c2].get("test_mae", float("nan")))
    after_key_mae = f"after_{c2}_mae"
    c1_after_c2_mae = float(res[c1].get(after_key_mae, float("nan")))

    forgetting_abs = c1_after_c2_mae - c1_test_mae
    forgetting_rel = (forgetting_abs / c1_test_mae * 100.0) if c1_test_mae == c1_test_mae else float("nan")

    rows.append({
        "exp": name,
        "preset": meta["preset"],
        "method": meta["method"],
        "lambda": meta["lambda"] if meta["method"] == "ewc" else None,
        "encoder": meta["encoder"],
        "seed": int(meta["seed"]) if meta["seed"] is not None else None,
        "c1_name": c1,
        "c2_name": c2,
        "c1_mae": c1_test_mae,
        "c1_after_c2_mae": c1_after_c2_mae,
        "c1_forgetting_mae_abs": forgetting_abs,
        "c1_forgetting_mae_rel_%": forgetting_rel,
        "c2_mae": c2_test_mae,
    })

df = pd.DataFrame(rows)

# Asegura columnas numéricas auxiliares
if "lambda_num" not in df.columns:
    df["lambda_num"] = pd.to_numeric(df["lambda"], errors="coerce")  # '1e+08' -> 1e+08 ; NAIVE -> NaN

# Deja 'seed' como entero y elimina 'seed_num' si existe
df["seed"] = pd.to_numeric(df["seed"], errors="coerce").astype("Int64")
if "seed_num" in df.columns:
    df = df.drop(columns=["seed_num"])

# Ordenar: preset, method, encoder, lambda_num (NaN al final), seed
df = df.sort_values(
    by=["preset", "method", "encoder", "lambda_num", "seed"],
    na_position="last",
    ignore_index=True,
)

df


,exp,preset,method,lambda,encoder,seed,c1_name,c2_name,c1_mae,c1_after_c2_mae,c1_forgetting_mae_abs,c1_forgetting_mae_rel_%,c2_mae,lambda_num
0,continual_accurate_ewc_lam_1e+06_rate_seed_42,accurate,ewc,1e+06,rate,42,circuito1,circuito2,0.133648,0.337355,0.203707,152.420925,0.220824,1.000000e+06
1,continual_accurate_ewc_lam_1e+06_rate_seed_43,accurate,ewc,1e+06,rate,43,circuito1,circuito2,0.145883,0.141756,-0.004126,-2.828558,0.240581,1.000000e+06
2,continual_accurate_ewc_lam_3e+06_rate_seed_42,accurate,ewc,3e+06,rate,42,circuito1,circuito2,0.133648,0.342694,0.209046,156.415913,0.246550,3.000000e+06
3,continual_accurate_ewc_lam_3e+06_rate_seed_43,accurate,ewc,3e+06,rate,43,circuito1,circuito2,0.145883,0.140616,-0.005267,-3.610151,0.258573,3.000000e+06
4,continual_accurate_ewc_lam_1e+07_rate_seed_42,accurate,ewc,1e+07,rate,42,circuito1,circuito2,0.133648,0.150086,0.016438,12.299507,0.258577,1.000000e+07
5,continual_accurate_ewc_lam_1e+07_rate_seed_43,accurate,ewc,1e+07,rate,43,circuito1,circuito2,0.145883,0.140613,-0.005270,-3.612234,0.263319,1.000000e+07
6,continual_accurate_ewc_lam_3e+07_rate_seed_42,accurate,ewc,3e+07,rate,42,circuito1,circuito2,0.148162,0.138012,-0.010150,-6.850842,0.553535,3.000000e+07
7,continual_accurate_ewc_lam_1e+08_rate_seed_42,accurate,ewc,1e+08,rate,42,circuito1,circuito2,0.148162,0.143491,-0.004672,-3.153023,0.719766,1.000000e+08
8,continual_accurate_naive_rate_seed_42,accurate,naive,None,rate,42,circuito1,circuito2,0.134141,0.257095,0.122953,91.659523,0.178753,NaN
9,continual_accurate_naive_rate_seed_43,accurate,naive,None,rate,43,circuito1,circuito2,0.145045,0.238982,0.093937,64.763894,0.175078,NaN


In [ ]:
# ====================== Vista agregada (media±std por preset/method/λ/encoder) ======================
import pandas as pd

# Métricas a agregar
cols_metrics = ["c1_mae", "c1_after_c2_mae", "c1_forgetting_mae_abs", "c1_forgetting_mae_rel_%", "c2_mae"]

# Copia y asegura columna numérica auxiliar para ordenar por λ
gdf = df.copy()
if "lambda_num" not in gdf.columns:
    gdf["lambda_num"] = pd.to_numeric(gdf["lambda"], errors="coerce")  # NA para NAIVE

# Agregación: media, std y número de corridas (semillas) por combinación
agg = (
    gdf
    .groupby(["preset", "method", "encoder", "lambda", "lambda_num"], dropna=False)[cols_metrics]
    .agg(["mean", "std", "count"])
    .reset_index()
)

# Aplanar nombres de columnas (de MultiIndex a una sola capa)
agg.columns = [
    "_".join(filter(None, map(str, col))).rstrip("_")
    for col in agg.columns.to_flat_index()
]

# Ordena por preset/method/encoder/λ_num (NaN al final ⇒ NAIVE al final de su grupo)
agg = agg.sort_values(
    by=["preset", "method", "encoder", "lambda_num"],
    na_position="last",
    ignore_index=True,
)

# (Opcional) guardar a CSV
summary_dir = ROOT / "outputs" / "summary"
summary_dir.mkdir(parents=True, exist_ok=True)
agg.to_csv(summary_dir / "continual_summary_agg.csv", index=False)
print("Guardado:", summary_dir / "continual_summary_agg.csv")

agg


In [ ]:
# ====================== Formateo para la memoria (tabla compacta) ======================

def fmt(x, prec=4):
    # Redondea y gestiona NaN de forma amigable
    import pandas as pd
    return "" if pd.isna(x) else f"{x:.{prec}f}"

show = agg.copy()

# 1) Crea 'count' a partir de cualquiera de las columnas *_count
count_cols = [c for c in show.columns if c.endswith("_count")]
if count_cols:
    show["count"] = show[count_cols[0]].astype("Int64")  # todas deberían coincidir
    # (opcional) elimina las columnas *_count individuales
    show = show.drop(columns=count_cols)

# 2) Redondea columnas de medias/desviaciones
for c in [c for c in show.columns if c.endswith("_mean") or c.endswith("_std")]:
    show[c] = show[c].map(lambda v: fmt(v, 4))

# 3) Selección de columnas clave (ajusta el orden a tu gusto)
cols = [
    "preset", "method", "encoder", "lambda",
    "c1_mae_mean", "c1_forgetting_mae_rel_%_mean", "c2_mae_mean",
    "c1_mae_std",  "c1_forgetting_mae_rel_%_std",  "c2_mae_std",
    "count"
]

# Si alguna columna no existiera (según tus métricas), la ignoramos con aviso
missing = [c for c in cols if c not in show.columns]
if missing:
    print("Aviso: faltan columnas en 'show':", missing)
    cols = [c for c in cols if c in show.columns]

show = show[cols].rename(columns={
    "preset": "preset",
    "method": "método",
    "encoder": "codificador",
    "lambda": "λ",
    "c1_mae_mean": "MAE Tarea1 (media)",
    "c1_forgetting_mae_rel_%_mean": "Olvido T1 (%) (media)",
    "c2_mae_mean": "MAE Tarea2 (media)",
    "c1_mae_std": "MAE Tarea1 (σ)",
    "c1_forgetting_mae_rel_%_std": "Olvido T1 (%) (σ)",
    "c2_mae_std": "MAE Tarea2 (σ)",
    "count": "n (semillas)"
})

show
